In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# Load data
blends_data = pd.read_csv('/Users/huiyisang/Desktop/blending_blends.csv')
materials_data = pd.read_csv('/Users/huiyisang/Desktop/blending_materials.csv')

# Get dimensions
num_blends = blends_data.shape[0]  # 50 blends
num_materials = materials_data.shape[0]  # 100 materials

# Extract data from files
demands = blends_data['demand'].values
quality_min = blends_data['quality_min'].values
quality_max = blends_data['quality_max'].values

availability = materials_data['availability'].values
costs = materials_data['cost'].values
p_max = materials_data['p_max'].values

# Extract quality contribution matrix (50x100)
quality_columns = [f'quality_{j+1}' for j in range(num_materials)]
quality_contribution = blends_data[quality_columns].values

# Create model - now using a nonlinear program
model = gp.Model("EcoClean_Nonlinear_Blending")

# Create variables - x[i,j] is the amount of material j used in blend i
x = model.addVars(num_blends, num_materials, vtype=GRB.CONTINUOUS, name="x")

# Set objective - minimize total cost
objective = gp.quicksum(x[i, j] * costs[j] for i in range(num_blends) for j in range(num_materials))
model.setObjective(objective, GRB.MINIMIZE)

# Constraint 1: Meet demand for each blend
for i in range(num_blends):
    model.addConstr(gp.quicksum(x[i, j] for j in range(num_materials)) == demands[i], 
                    f"demand_constraint_{i}")

# Constraint 2: Raw material availability
for j in range(num_materials):
    model.addConstr(gp.quicksum(x[i, j] for i in range(num_blends)) <= availability[j], 
                    f"availability_constraint_{j}")

# Constraint 3: Quality constraints for each blend
for i in range(num_blends):
    # Calculate the average quality for blend i
    blend_quality = gp.quicksum(x[i, j] * quality_contribution[i, j] 
                              for j in range(num_materials)) / demands[i]
    
    # Enforce quality bounds
    model.addConstr(blend_quality >= quality_min[i], f"min_quality_constraint_{i}")
    model.addConstr(blend_quality <= quality_max[i], f"max_quality_constraint_{i}")

# Constraint 4: Modified Maximum proportion constraint - NONLINEAR
for i in range(num_blends):
    for j in range(num_materials):
        # Sum of squared values of raw materials
        squared_sum = gp.quicksum(x[i, k] * x[i, k] for k in range(num_materials))
        
        # Add the nonlinear constraint
        model.addConstr(x[i, j] <= p_max[j] * squared_sum, 
                       f"nonlinear_proportion_constraint_{i}_{j}")

# Set nonlinear parameter to handle the nonlinear constraints
model.params.NonConvex = 2

# Optimize the model
model.optimize()

# Print the optimal objective value
if model.status == GRB.OPTIMAL:
    print(f"Minimum production cost for the nonlinear program: ${model.objVal:.2f}")
else:
    print(f"Optimization status: {model.status}")

Set parameter Username
Set parameter LicenseID to value 2610034
Academic license - for non-commercial use only - expires 2026-01-14
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 250 rows, 5000 columns and 20000 nonzeros
Model fingerprint: 0xfff6f266
Model has 5000 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  QMatrix range    [3e-01, 7e-01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+04]
Presolve removed 50 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 4995 rows and 50 columns
Presolve time: 0.05s
Presolved: 10295 rows, 10051 columns, 44455 nonzeros
Presolved model has 5000 bilinear constraint(s)
